# Delisted & Suspended Stocks Filtering

* Price Loader에서 Filtering을 해줄 수 있지만 추가로 정보를 보려면 KRX 공개 정보를 확인하면 된다.

In [1]:
import logging, os, pickle
import requests, glob
from datetime import datetime
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import time
from datetime import date
import urllib.request
from selenium.webdriver import Chrome
import json, re, sys, h5py
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt
import pymysql
import matplotlib.pyplot as plt
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from pykrx import stock
import warnings
warnings.filterwarnings(action='ignore')
import shutil
from matplotlib.pyplot import cm
import numpy as np
import scipy.stats as stats
from scipy import stats
import pyautogui
from statsmodels.tsa.stattools import coint, adfuller
from sklearn.linear_model import LinearRegression
import matplotlib.font_manager as fm
import sys
import seaborn as sns
%matplotlib inline
plt.rcParams['font.family'] = 'NanumSquare_ac'
sys.path.append('C:\\Users\Bae Kyungmo\OneDrive\Desktop\StockTraidingAutomation\DataModeling')
from Loader_v2 import Loader
ldr = Loader(pwd='z')

In [11]:
files = glob.glob("../FullCache/*.csv")
files

['../FullCache\\매매거래정지종목_2021.csv', '../FullCache\\상장폐지종목_2010_2021.csv']

In [13]:
def DelistTransfer(df):
    transfer = df.copy()
    code_format = []
    for e in transfer['이전상장후_종목코드'].values:
        try :
            code_format.append('{:06d}'.format(int(e)))
        except :
            code_format.append(np.nan)
    transfer['이전상장후_종목코드'] = code_format
    transfer.drop(['번호','증권구분','주식종류','업종','액면가','상장주식수'],axis=1,inplace=True)
    transfer['상장일'] = transfer['상장일'].map(lambda x : x.replace('/','-'))
    transfer['폐지일'] = transfer['폐지일'].map(lambda x : x.replace('/','-'))
    transfer['정리매매기간_시작일'] = transfer['정리매매기간_시작일'].map(lambda x : x.replace('/','-') if type(x)==str else x)
    transfer['정리매매기간_종료일'] = transfer['정리매매기간_종료일'].map(lambda x : x.replace('/','-') if type(x)==str else x)
    return transfer

In [14]:
delist = pd.read_csv(files[1],encoding='cp949')
qq = DelistTransfer(delist)

In [197]:
def DelistComplement():
    pr1 = ldr.GetPricelv1('2010-01-01','2021-03-10')
    pr1 = pr1[lambda x : x['volume']==0]
    dl_codes = list(set(pr1.CODE.values))
    delist_dict = {}
    for cd in dl_codes:
        tmp = pr1[pr1.CODE==cd].sort_values(by=['DATE'])
        delist_dict[cd] = tmp.DATE.to_list()
    return delist_dict

In [198]:
%%time
rst = DelistComplement()

Wall time: 41.9 s
